# 1. Data collection
Sound of cars and trams are partly recorded and used during the trainingm and parts of them are from the Internet.

For the recorded sounds:
- source: trams and cars passing in Hervanta
- what: the sound of trams arriving to the platform and cars passing
- where: around Hervanta, particularly Opiskelija tram platform
- method: a smartphone with audio recording software were used

For the online sourced sounds:
1. Tram
- source: [tram](https://freesound.org/people/ilknur_bas/packs/37103/)
- what: the sound of trams arriving to the platform and cars passing
- where: around Tampere, Kaleva
- method: an iPhone12 audio recording software were used
2. Car
- source: [car](https://freesound.org/people/trung1309vn/packs/39612/)
- what: the sound of cars A passing by from distance
- where: Hervannan valtaväylä road, Tampere
- method: a Redmi 9C audio recording software were used

In [2]:
import librosa as lb
import sounddevice as sd
import soundfile as sf
data, sr = lb.load('data/tram/tram1.wav', sr=None)
sd.play(data)